# Notebook for prototyping

In [1]:
import pandas as pd
import polars as pl
import pickle
import xgboost as xgb
import optuna
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from lets_plot import *
import typing as t

LetsPlot.setup_html()

In [2]:
# Define file paths
mapping_path = '../data/feature_mapping_train.pkl'
features_path = '../data/train_data_features.feather'
target_path = '../data/train_data_target.feather'

# Load the mapping (pickle file)
with open(mapping_path, 'rb') as f:
    mapping = pickle.load(f)

# Load train_features and train_target from Feather files
train_features = pl.read_ipc(features_path)  # Polars uses `read_ipc` for Feather files
train_target = pl.read_ipc(target_path)

# Convert the mapping (dictionary or list) to a Polars DataFrame
feature_mapping = pl.DataFrame(mapping)

In [3]:
def create_calendric_features(df: pl.DataFrame, date_column: str) -> pl.DataFrame:
    """
    Create calendric features for a given Polars DataFrame with a date column.

    Parameters:
        df (pl.DataFrame): Input Polars DataFrame containing a date column.
        date_column (str): Name of the column containing dates.

    Returns:
        pl.DataFrame: Polars DataFrame with added calendric features.
    """
    # Ensure the date column is in datetime format
    df = df.with_columns(pl.col(date_column).cast(pl.Date).alias(date_column))

    # Create basic calendric features
    df = df.with_columns([
        pl.col(date_column).dt.month().alias("month"),
        (pl.col(date_column).dt.weekday()).alias("day_of_week"),  # Monday=1, ..., Sunday=7
        (pl.col(date_column).dt.strftime("%V").cast(pl.Int32)).alias("week_of_year"),  # ISO week number
        (pl.col(date_column).dt.year()).alias("year"),
        (pl.col(date_column).dt.weekday() >= 5).alias("is_weekend")  # True for Saturday/Sunday
    ])

    # Create the "quarter" column based on the "month" column
    df = df.with_columns(((pl.col("month") - 1) // 3 + 1).alias("quarter"))

    return df

def add_lag_features(
    df: pl.DataFrame,
    lags: t.Union[int, t.List[int], range] = 1,
    group_by_cols: t.List[str] = None,
    value_col: str = "value",
    date_col: str = "date",
    
) -> pl.DataFrame:
    """
    Add lag features to a DataFrame.

    Parameters
    ----------
    df : pl.DataFrame
        Input DataFrame containing time series data.
    value_col : str, default "value"
        Name of the column to create lag features for.
    group_by_cols : List[str], optional
        List of columns to group by when creating lags. If None, defaults to 
        ["skuID", "frequency"] if both exist, otherwise ["skuID"].
    date_col : str, default "date"
        Name of the date column used for sorting.
    lags : Union[int, List[int], range], optional
        Lag periods to create. If int, creates lags from 1 to that number.
        If List[int] or range, creates lags for those specific values.
        If None, defaults to range(1, 8).

    Returns
    -------
    pl.DataFrame
        DataFrame with added lag columns named "{value_col}_lag_{lag}".
    """
    
    # Handle default parameters
    if group_by_cols is None:
        group_by_cols = ["skuID", "frequency"]
    
    sort_cols = group_by_cols + [date_col]
    
    # Validate that required columns exist
    missing_cols = [col for col in sort_cols + [value_col] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in DataFrame: {missing_cols}")
    
    # Sort the DataFrame
    df_sorted = df.sort(sort_cols)
    
    # Create lag features
    lag_features = [
        pl.col(value_col).shift(lag).over(group_by_cols).alias(f"{value_col}_lag_{lag}")
        for lag in lags
    ]
    
    # Add lag columns to the DataFrame
    result = df_sorted.with_columns(lag_features)
    
    return result

In [ ]:
#diff_features
df = df.with_columns([
    # First-order difference: y_t - y_{t-1}
    (pl.col("target") - pl.col("target").shift(1)).alias("diff_1"),

    # Second-order difference: (y_t - y_{t-1}) - (y_{t-1} - y_{t-2})
    (
        (pl.col("target") - pl.col("target").shift(1)) -
        (pl.col("target").shift(1) - pl.col("target").shift(2))
    ).alias("diff_2")
])

In [4]:
#Feature engineering
df = create_calendric_features(train_features, 'date')
df = df.to_dummies(
    columns=["day_of_week", "month", "quarter", "week_of_year", "year", "is_weekend"]
)
df = add_lag_features(
    df,
    lags=range(1, 8),
    group_by_cols=["skuID", "frequency"],
    value_col="feature_0038",
    date_col="date"
)
df = df.drop('lag_target_1','feature_0038')
df = df.filter(pl.col("not_for_sale") != 1)


In [6]:
earliest = df["date"].min()
latest = df["date"].max()

# Create a date range from earliest to latest (inclusive)
date_range = pl.date_range(earliest, latest, "1d", eager=True)

# Create the trend column: count from 1 to N
trend = pl.int_range(1, len(date_range) + 1, eager=True)

# Create the new DataFrame
result = pl.DataFrame({
    "date": date_range,
    "trend": trend
})

df = df.join(result, on="date", how="left")

In [7]:
df.shape

(46881677, 144)

In [152]:
(
    df.lazy()
    .select(["feature_0038_lag_1", "productID"])
    .sort("feature_0038_lag_1")
    .drop_nulls()
    .group_by("productID")
    .agg(pl.col("feature_0038_lag_1").mean().alias("mean"))
    .sort("mean")
    .collect()
)

productID,mean
i64,f64
78778,0.036712
79731,0.053758
79427,0.055439
79382,0.056939
79617,0.05807
…,…
81055,20.714691
81023,25.647474
80720,29.551907


In [9]:
#verschiedene produkte samplen
(df.lazy()
 .select("productID")
 .unique()
 .collect()
 .sample(n=10,seed=42))

productID
i64
79613
79697
79274
79847
79406
78524
79477
78809
79259


In [ ]:
X = (df
     .lazy()
     .filter(pl.col('productID') == 80558)
     .drop_nulls()
     .sort("date","skuID")
     .collect())

X = (X
     .hstack(X
             .select("skuID")
             .to_dummies()))

X_bdIDs = (X
           .lazy()
           .select('bdID')
           .unique()
           .collect()
           .to_numpy()
           .flatten())

y = (train_target
     .lazy()
     .filter(pl.col("bdID").is_in(X_bdIDs))
     .join(
         X.lazy().select("bdID","skuID"), 
         on="bdID", 
         how="left")
     .sort("date","skuID")
     .collect())

In [28]:
#split 80/20 into train and test sets
X_t, X_v, y_t, y_v = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [36]:
X_t.columns

['frequency',
 'idx',
 'bdID',
 'base_date',
 'date',
 'dateID',
 'skuID',
 'productID',
 'storeID',
 'companyID',
 'missing_value',
 'not_for_sale',
 'feature_0000',
 'feature_0001',
 'feature_0002',
 'feature_0003',
 'feature_0004',
 'feature_0005',
 'feature_0006',
 'feature_0007',
 'feature_0008',
 'feature_0009',
 'feature_0010',
 'feature_0011',
 'feature_0012',
 'feature_0013',
 'feature_0014',
 'feature_0015',
 'feature_0016',
 'feature_0017',
 'feature_0018',
 'feature_0019',
 'feature_0020',
 'feature_0021',
 'feature_0022',
 'feature_0023',
 'feature_0024',
 'feature_0025',
 'feature_0026',
 'feature_0027',
 'feature_0028',
 'feature_0029',
 'feature_0030',
 'feature_0031',
 'feature_0032',
 'feature_0033',
 'feature_0034',
 'feature_0035',
 'feature_0036',
 'feature_0037',
 'feature_0039',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'day_of_week_1',
 'day_of_week_2',
 'da

In [29]:
FEATURE_COLUMNS = X.columns[12:]
TARGET_COLUMN = y.columns[-2]

In [150]:
def objective(trial):
    # Focus on the most impactful hyperparameters (80/20 approach)
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.5, log=True),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
        'random_state': 42
    }
    
    # Time series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    mse_scores = []

    for train_idx, val_idx in tscv.split(X_t):
        # Split the data into training and validation sets
        X_train = X_t[train_idx].sort("date","skuID").select(FEATURE_COLUMNS).to_numpy()
        y_train = y_t[train_idx].sort("date","skuID").select(TARGET_COLUMN).to_numpy()
        X_val = X_t[val_idx].sort("date","skuID").select(FEATURE_COLUMNS).to_numpy()
        y_val = y_t[val_idx].sort("date","skuID").select(TARGET_COLUMN).to_numpy()

        # Train the model
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_val)
        y_pred = y_pred.round().astype(int)

        # Evaluate the model
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)

    # Return the average MSE across all folds
    return np.mean(mse_scores)

# Create study and optimize
print("Starting hyperparameter optimization...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials = 50)  # 50 trials for good balance of speed/performance

# Get best parameters
best_params = study.best_params
print(f"\nBest parameters found:")
for key, value in best_params.items():
    print(f"{key}: {value}")

# Train final model with best parameters
print(f"\nBest MSE: {study.best_value:.4f}")
print("Training final model with best parameters...")

final_model = xgb.XGBRegressor(**best_params)
final_model.fit(
    X_t.select(FEATURE_COLUMNS).to_numpy(),
    y_t.select(TARGET_COLUMN).to_numpy()
)

# Make final predictions on the validation set
y_pred_final = final_model.predict(X_v.select(FEATURE_COLUMNS).to_numpy())
y_pred_final = y_pred_final.round().astype(int)

# Evaluate final model
mse_final = mean_squared_error(y_v.select(TARGET_COLUMN).to_numpy(), y_pred_final)
r2_final = r2_score(y_v.select(TARGET_COLUMN).to_numpy(), y_pred_final)

print(f'\nFinal Model Performance:')
print(f'MSE: {mse_final:.4f}')
print(f'R²: {r2_final:.4f}')

[I 2025-07-20 21:44:25,393] A new study created in memory with name: no-name-ee6b6e51-75b4-46e2-b8c6-000ac2f3ea3d


Starting hyperparameter optimization...


[I 2025-07-20 21:44:26,757] Trial 0 finished with value: 1737.8665632273078 and parameters: {'n_estimators': 294, 'max_depth': 3, 'learning_rate': 0.2897025273428427, 'subsample': 0.9901452240495073, 'colsample_bytree': 0.935902793707244, 'reg_alpha': 5.208264718924659, 'reg_lambda': 7.359379460423019}. Best is trial 0 with value: 1737.8665632273078.
[I 2025-07-20 21:44:29,627] Trial 1 finished with value: 1644.4335919317302 and parameters: {'n_estimators': 80, 'max_depth': 18, 'learning_rate': 0.0939272492901301, 'subsample': 0.7210935913541182, 'colsample_bytree': 0.8023888648290669, 'reg_alpha': 5.392748839392548, 'reg_lambda': 1.0321986010022355}. Best is trial 1 with value: 1644.4335919317302.
[I 2025-07-20 21:44:31,510] Trial 2 finished with value: 2033.3333591931732 and parameters: {'n_estimators': 58, 'max_depth': 18, 'learning_rate': 0.4133188436434523, 'subsample': 0.765621633079961, 'colsample_bytree': 0.9636457349455524, 'reg_alpha': 7.091026352980658, 'reg_lambda': 2.23456


Best parameters found:
n_estimators: 70
max_depth: 3
learning_rate: 0.09604760376186887
subsample: 0.9983713096682683
colsample_bytree: 0.9862453793161666
reg_alpha: 5.448559030263386
reg_lambda: 9.313894097480286

Best MSE: 1296.2871
Training final model with best parameters...

Final Model Performance:
MSE: 448.1603
R²: 0.7156


In [ ]:
3000/60/

16.666666666666668

In [151]:
# Extract optimization history from Optuna study
optimization_data = {
    "Trial": list(range(len(study.trials))),  # Trial numbers
    "MSE": [trial.value for trial in study.trials]  # MSE values for each trial
}

# Convert to pandas DataFrame for plotting
optimization_df = pd.DataFrame(optimization_data)

# Create the plot
plt = ggplot(optimization_df, aes(x="Trial", y="MSE")) + \
    geom_line(size=1, color="blue") + \
    geom_smooth(method='lm', color="red", size=1, linetype="dashed") + \
    ggtitle("Optuna Optimization Progress") + \
    xlab("Trial") + \
    ylab("MSE") + \
    theme_minimal() + \
    ggsize(900, 500) 

# Show the plot
plt.show()

In [34]:
preds = final_model.predict(X_v.select(FEATURE_COLUMNS).to_numpy())
preds = preds.round().astype(int)

data = X_v.hstack(pl.DataFrame({"prediction": preds}))
data = data.hstack(pl.DataFrame({"target": y_v.select(TARGET_COLUMN)}))

In [35]:
mse_final = mean_squared_error(data.select(TARGET_COLUMN).to_numpy(), preds)
r2_final = r2_score(data.select(TARGET_COLUMN).to_numpy(), preds)

print(f'\nFinal Model Performance:')
print(f'MSE: {mse_final:.4f}')
print(f'R²: {r2_final:.4f}')


Final Model Performance:
MSE: 451.8591
R²: 0.7132


In [155]:
#aggregated forecast -> mean prediction and target per date

mondays = (
    data
    .filter((pl.col("day_of_week_1") == 1))
    .select("date")
    .to_series()
    .to_list()
)

vline_df = pd.DataFrame({
    "x": pd.to_datetime(mondays),
    "type": ["Week Start"] * len(mondays)
})

plotdata = (
    data
    .group_by("date")
    .agg(
        pl.col("prediction").mean().alias("mean_prediction"),
        pl.col("target").mean().alias("mean_target")
     )
    .sort("date")
    .unpivot(
            on =["mean_prediction", "mean_target"],
            index = "date",
            variable_name = "type",
            value_name = "value"
           )
    .to_pandas()
)

p = ggplot(plotdata) + \
    geom_line(aes(x='date', y='value', color='type'), alpha=0.7) + \
    geom_vline(data=vline_df, mapping=aes(xintercept='x', color='type'),
                linetype='dashed', alpha=0.5, size=0.5, show_legend=True) + \
    scale_color_manual(values={
        'mean_target': '#e41a1c',
        'mean_prediction': '#377eb8',
        'Week Start': 'grey'
    }, labels={
        'mean_target': 'Target',
        'mean_prediction': 'Prediction',
        'Week Start': 'Week Start'
    }) + \
    labs(title='t+1 Forecast Using Product-Level XGBoost: Mean Prediction and Target per Date (Aggregated Across SKUs)',
         x='Date', y='Value',
         color='Legend') + \
    ggsize(2800, 600) + \
    theme_minimal() + \
        theme(
        plot_title=element_text(hjust=0.5),
        panel_grid_major_x=element_blank(),
        panel_grid_minor_x=element_blank(),
        panel_grid_major_y=element_blank(),
        panel_grid_minor_y=element_blank()
    )

p.show()

In [1]:
#aggregated forecast -> sum prediction and target per date

mondays = (
    data
    .filter((pl.col("day_of_week_1") == 1))
    .select("date")
    .to_series()
    .to_list()
)

vline_df = pd.DataFrame({
    "x": pd.to_datetime(mondays),
    "type": ["Week Start"] * len(mondays)
})

plotdata = (
    data
    .group_by("date")
    .agg(
        pl.col("prediction").sum().alias("sum_prediction"),
        pl.col("target").sum().alias("sum_target")
     )
    .sort("date")
    .unpivot(
            on =["sum_prediction", "sum_target"],
            index = "date",
            variable_name = "type",
            value_name = "value"
           )
    .to_pandas()
)

p = ggplot(plotdata) + \
    geom_line(aes(x='date', y='value', color='type'), alpha=0.7) + \
    geom_vline(data=vline_df, mapping=aes(xintercept='x', color='type'),
                linetype='dashed', alpha=0.5, size=0.5, show_legend=True) + \
    scale_color_manual(values={
        'sum_target': '#e41a1c',
        'sum_prediction': '#377eb8',
        'Week Start': 'grey'
    }, labels={
        'sum_target': 'Target',
        'sum_prediction': 'Prediction',
        'Week Start': 'Week Start'
    }) + \
    labs(title='t+1 Forecast Using Product-Level XGBoost: Sum Prediction and Target per Date (Aggregated Across SKUs)',
         x='Date', y='Value',
         color='Legend') + \
    ggsize(3000, 600) + \
    theme_minimal() + \
        theme(
        plot_title=element_text(hjust=0.5),
        panel_grid_major_x=element_blank(),
        panel_grid_minor_x=element_blank(),
        panel_grid_major_y=element_blank(),
        panel_grid_minor_y=element_blank()
    )

p.show()

NameError: name 'data' is not defined

In [157]:

sku_ids = data.select("skuID").unique().to_numpy().flatten().tolist()

for sku in sku_ids:
    # Mondays for the current SKU only
    mondays = (
        data
        .filter((pl.col("day_of_week_1") == 1) & (pl.col("skuID") == sku))
        .select("date")
        .to_series()
        .to_list()
    )

    # Plot data (melted from wide to long)
    plotdata = (
        data
        .filter(pl.col("skuID") == sku)
        .unpivot(
                on =["prediction", "target"],
                index = "date",
                variable_name = "type",
                value_name = "value"
           )
       .to_pandas()
    )

    if plotdata.empty:
        continue  # Skip if no data for this SKU

    # Vertical line data as DataFrame with "type" for legend
    vline_df = pd.DataFrame({
        "x": pd.to_datetime(mondays),
        "type": ["Week Start"] * len(mondays)
    })

    # Build plot
    p = ggplot(plotdata) + \
        geom_line(aes(x='date', y='value', color='type'), alpha=0.7) + \
        geom_vline(data=vline_df, mapping=aes(xintercept='x', color='type'),
                   linetype='dashed', alpha=0.5, size=0.5, show_legend=True) + \
        scale_color_manual(values={
            'target': '#e41a1c',
            'prediction': '#377eb8',
            'Week Start': 'grey'
        }, labels={
            'target': 'Target',
            'prediction': 'Prediction',
            'Week Start': 'Week Start'
        }) + \
        labs(title='t+1 Forecast Using Product-Level XGBoost: Prediction and Target per Date (for SKU {})'.format(sku),
             x='Date', y='Value', color='Legend') + \
        ggsize(2800, 600) + \
        theme_minimal() + \
        theme(
            plot_title=element_text(hjust=0.5),
            panel_grid_major_x=element_blank(),
            panel_grid_minor_x=element_blank(),
            panel_grid_major_y=element_blank(),
            panel_grid_minor_y=element_blank()
        )

    p.show()


In [ ]:
#erstele boxplots der fehler
data_boxplots =(
    data
    .select("date","prediction","target", "skuID")
    .with_columns((pl.col("target") - pl.col("prediction")).alias("error"))
    .with_columns(pl.col("skuID").cast(pl.String).alias("skuID"))
)

In [149]:
p = ggplot(data_boxplots.to_pandas(), aes(x='skuID', y='error')) + \
    geom_boxplot(aes(group='skuID'), outlier_color='red', outlier_size=0.5) + \
    labs(title='Boxplot of Prediction Errors per SKU (Target - Prediction)',
         x='SKU ID',
         y='Prediction Error')+ \
    theme_minimal() + \
    ggsize(1000, 400) 

p.show()

In [ ]:
#funktioniert, ich muss nicht für einzelne SKUs predicten

prediction_data = {}
for i in sku_ids:
    mse = mean_squared_error(
        data.filter(pl.col("skuID") == i).sort("date", "skuID").select(TARGET_COLUMN).to_numpy(),
        data.filter(pl.col("skuID") == i).sort("date", "skuID").select("prediction").to_numpy()
    )
    r2 = r2_score(
        data.filter(pl.col("skuID") == i).sort("date", "skuID").select(TARGET_COLUMN).to_numpy(),
        data.filter(pl.col("skuID") == i).sort("date", "skuID").select("prediction").to_numpy()
    )
    prediction_data[i] = (
        data.filter(pl.col("skuID") == i),
        {"MSE": mse, "R²": r2}
    )
    print(f'\nFinal Model Performance for SKU {i}:')
    print(f'MSE: {mse:.4f}')
    print(f'R²: {r2:.4f}')



Final Model Performance for SKU 267030:
MSE: 1406.2455
R²: 0.5956

Final Model Performance for SKU 270079:
MSE: 132.7959
R²: 0.4895

Final Model Performance for SKU 273128:
MSE: 335.0052
R²: 0.5599

Final Model Performance for SKU 276177:
MSE: 434.2997
R²: 0.5842

Final Model Performance for SKU 279226:
MSE: 544.8217
R²: 0.5633

Final Model Performance for SKU 282275:
MSE: 104.2222
R²: 0.3782

Final Model Performance for SKU 285324:
MSE: 136.4444
R²: 0.3436

Final Model Performance for SKU 288373:
MSE: 615.4755
R²: 0.6434

Final Model Performance for SKU 260932:
MSE: 499.5155
R²: 0.5495

Final Model Performance for SKU 263981:
MSE: 309.5207
R²: 0.5742


In [62]:
prediction_data = {}

# Loop through each skuID
for sku in sku_ids:
    # Filter X_v for the current skuID
    X_sku = X_v.filter(pl.col("skuID") == sku)
    y_sku = y_v.join(X_sku.select("skuID","bdID"), on="bdID")
    
    #sort X_sku and y_sku by date
    X_sku = X_sku.sort("date")
    y_sku = y_sku.sort("date")

    # Make prediction using the model
    preds = final_model.predict(X_sku.select(FEATURE_COLUMNS).to_numpy())
    preds = preds.round().astype(int)


    # Store predictions in dictionary
    prediction_data[sku] = [X_sku, y_sku, preds]